In [3]:
import pandas as pd
from urllib.parse import urlencode
import requests
from requests import Response
import plotly.graph_objects as go
import sqlalchemy
from datetime import datetime, date, time
import os, sys
import datetime as dt
from dateutil.relativedelta import relativedelta
from math import ceil
import numpy as np
import foward_test_valuation_strategy as common

In [4]:
keys = ['total equity', 'net accounting profit/(loss) before tax', 'interest expenses', 'fixed assets', 
        'profit/(loss) before tax', 'profit before tax']

hose_list = ['AAA','AMD','ANV','APG','ASM','AST','BCG','BID','BMP','BWE','CCL','CII','CMG','CRE','CSV','CTD','CTF',
             'CTG','CTI','CVT','D2D','DAG','DBC','DBD','DCL','DCM','DGW','DHC','DHG','DIG','DLG','DPG','DPM','DRC',
             'DRH','DXG','EIB','FCN','FIR','FIT','FLC','FPT','FRT','FTS','GAS','GEG','GEX','GIL','GMD','HAG','HAI',
             'HAX','HBC','HCM','HDB','HDC','HDG','HHS','HII','HNG','HPG','HPX','HQC','HSG','HT1','HVH','HVN','IBC',
             'IDI','IJC','IMP','ITA','JVC','KBC','KDH','KOS','KSB','LCG','LDG','LHG','MBB','MSH','MSN','MWG','NCT',
             'NKG','NLG','NT2','NTL','NVL','OGC','PC1','PDR','PHC','PHR','PLX','PME','PNJ','POW','PPC','PTB','PVD',
             'PVT','REE','ROS','SAB','SAM','SBT','SCR','SHI','SKG','SMB','SSI','STB','STK','SZC','SZL','TCB','TCH',
             'TCM','TDH','TDM','TLG','TNA','TNI','TPB','TTB','TVB','VCB','VCI','VGC','VHC','VHM','VIC','VJC','VND',
             'VNG','VNM','VPB','VPG','VPI','VRE','VSC']

In [5]:
len(hose_list)

143

In [ ]:
access_tok = common.login()

In [ ]:
data = pd.DataFrame()
for key in keys:
    for year in range(2014, 2021):
        for quarter in range(1, 5):
            df = common.query_financial_data(year=year, 
                                             quarter=quarter, 
                                             list_stock=hose_list, 
                                             type=key, 
                                             access_token=access_tok)
            if len(df) > 0:
                df = pd.melt(df, id_vars=['code', 'quarter', 'year'], value_vars=key)
            data = pd.concat([data, df])
            print('KEY: {} ====== YEAR: {} ===== QUAR: {}'.format(key, year, quarter))
    print('DONEEEEEEE')
data

In [ ]:
yearly_data = pd.DataFrame()
for key in keys:
    for year in range(2009, 2015):
        for quarter in range(1, 5):
            df = common.query_financial_data(year=year, 
                                             quarter=quarter, 
                                             list_stock=hose_list, 
                                             type=key, 
                                             access_token=access_tok)
            if len(df) > 0:
                df = pd.melt(df, id_vars=['code', 'quarter', 'year'], value_vars=key)
            yearly_data = pd.concat([yearly_data, df])
            print('KEY: {} ====== YEAR: {} ===== QUAR: {}'.format(key, year, quarter))
    print('DONEEEEEEE')
yearly_data

In [ ]:
data = pd.concat([yearly_data, data]).drop_duplicates()
data = data.drop_duplicates()

In [1]:
data.to_csv('financial_data_erp5.csv', index=None)

NameError: name 'data' is not defined

In [6]:
data = pd.read_csv('financial_data_erp5.csv').reset_index(drop=True)

In [7]:
equity = data[data['variable']=='total equity']
interest = data[data['variable']=='interest expenses']
asset = data[data['variable']=='fixed assets']
ebt = data[data['variable']=='net accounting profit/(loss) before tax']
ebt2 = data[data['variable']=='profit/(loss) before tax']
ebt3 = data[data['variable']=='profit before tax']

In [8]:
equity = equity.drop(columns=['variable']).rename(columns={'value':'total equity'})
interest = interest.drop(columns=['variable']).rename(columns={'value':'interest expenses'})
asset = asset.drop(columns=['variable']).rename(columns={'value':'fixed assets'})
ebt = ebt.drop(columns=['variable']).rename(columns={'value':'net accounting profit/(loss) before tax'})
ebt2 = ebt2.drop(columns=['variable']).rename(columns={'value':'profit/(loss) before tax'})
ebt3 = ebt3.drop(columns=['variable']).rename(columns={'value':'profit before tax'})

In [9]:
data = pd.merge(equity, interest, on=['code', 'quarter', 'year'], how='outer')
data = pd.merge(data, asset, on=['code', 'quarter', 'year'], how='outer')
data = pd.merge(data, ebt, on=['code', 'quarter', 'year'], how='outer')
data = pd.merge(data, ebt2, on=['code', 'quarter', 'year'], how='outer')
data = pd.merge(data, ebt3, on=['code', 'quarter', 'year'], how='outer')
data

,code,quarter,year,total equity,interest expenses,fixed assets,net accounting profit/(loss) before tax,profit/(loss) before tax,profit before tax
0,CTG,1,2011,22615899.0,NaN,3266595.0,NaN,NaN,1161832.0
1,KSB,1,2011,409344.0,NaN,165926.0,46876.0,NaN,NaN
2,PHC,1,2011,157942.0,-3048.0,87455.0,3089.0,NaN,NaN
3,STB,1,2011,14339855.0,NaN,3200199.0,NaN,NaN,715004.0
4,VCB,1,2011,26561802.0,NaN,1520908.0,NaN,NaN,1894534.0
...,...,...,...,...,...,...,...,...,...
4900,HDB,2,2015,NaN,NaN,NaN,NaN,NaN,172186.0
4901,TPB,3,2015,NaN,NaN,NaN,NaN,NaN,89330.0
4902,HDB,3,2015,NaN,NaN,NaN,NaN,NaN,63439.0
4903,HDB,4,2015,NaN,NaN,NaN,NaN,NaN,403464.0


In [10]:
data['net accounting profit/(loss) before tax'] = data['net accounting profit/(loss) before tax'].fillna(
                                                                                        data['profit before tax'])
data['net accounting profit/(loss) before tax'] = data['net accounting profit/(loss) before tax'].fillna(
                                                                                        data['profit/(loss) before tax'])

In [11]:
data.loc[data['code']==data['code'].shift(-1), 'total equity'] = data.loc[data['code']==data['code'].shift(-1), 
                                                                          'total equity'].fillna(method='ffill')
data.loc[data['code']==data['code'].shift(-1), 'fixed assets'] = data.loc[data['code']==data['code'].shift(-1), 
                                                                          'fixed assets'].fillna(method='ffill')

In [12]:
data = data.sort_values(by=['code', 'year', 'quarter']).reset_index(drop=True)

In [13]:
data = data.rename(columns={'net accounting profit/(loss) before tax': 'ebt'})

In [14]:
data['ebit'] = data['ebt'] + data['interest expenses']
data.loc[data['ebit'].isna(), 'ebit'] = data['ebt']

In [15]:
data = data.drop(columns=['profit/(loss) before tax', 'profit before tax'])

In [16]:
data['roic'] = data['ebit'] / (data['total equity']+data['fixed assets'])

In [17]:
data['qrt_yr'] = data['year'].apply(lambda x: str(x)) + data['quarter'].apply(lambda x: str(x))

In [18]:
data = data.drop_duplicates()

In [19]:
data

,code,quarter,year,total equity,interest expenses,fixed assets,ebt,ebit,roic,qrt_yr
0,AAA,1,2011,302752.0,-6887.0,356436.0,19161.0,12274.0,0.018620,20111
1,AAA,2,2011,311510.0,-8068.0,364849.0,11049.0,2981.0,0.004407,20112
2,AAA,3,2011,329784.0,-9747.0,405337.0,22561.0,12814.0,0.017431,20113
3,AAA,4,2011,350355.0,-10139.0,438773.0,25129.0,14990.0,0.018996,20114
4,AAA,1,2012,367052.0,-9031.0,430934.0,18266.0,9235.0,0.011573,20121
...,...,...,...,...,...,...,...,...,...,...
4900,VSC,4,2019,2070078.0,-552.0,1082398.0,110987.0,110435.0,0.035031,20194
4901,VSC,1,2020,2141761.0,-1075.0,1029230.0,81263.0,80188.0,0.025288,20201
4902,VSC,2,2020,2185842.0,-639.0,1016896.0,73815.0,73176.0,0.022848,20202
4903,VSC,3,2020,2175866.0,-89.0,965396.0,86117.0,86028.0,0.027386,20203


In [20]:
roic_yr = pd.pivot(data, index='qrt_yr', columns='code', values='roic')

In [21]:
roic_5yr_avg = roic_yr.rolling(20).mean().reset_index()
roic_5yr_avg = pd.melt(roic_5yr_avg, id_vars=['qrt_yr'], value_vars=roic_5yr_avg.columns[1:])
roic_5yr_avg = roic_5yr_avg.rename(columns={'value': 'roic_5yr_avg'})
roic_5yr_avg

,qrt_yr,code,roic_5yr_avg
0,20111,AAA,NaN
1,20112,AAA,NaN
2,20113,AAA,NaN
3,20114,AAA,NaN
4,20121,AAA,NaN
...,...,...,...
5715,20194,VSC,0.030265
5716,20201,VSC,0.029151
5717,20202,VSC,0.027461
5718,20203,VSC,0.025681


In [22]:
data = pd.merge(data, roic_5yr_avg, on=['code', 'qrt_yr'], how='inner')

In [23]:
data

,code,quarter,year,total equity,interest expenses,fixed assets,ebt,ebit,roic,qrt_yr,roic_5yr_avg
0,AAA,1,2011,302752.0,-6887.0,356436.0,19161.0,12274.0,0.018620,20111,NaN
1,AAA,2,2011,311510.0,-8068.0,364849.0,11049.0,2981.0,0.004407,20112,NaN
2,AAA,3,2011,329784.0,-9747.0,405337.0,22561.0,12814.0,0.017431,20113,NaN
3,AAA,4,2011,350355.0,-10139.0,438773.0,25129.0,14990.0,0.018996,20114,NaN
4,AAA,1,2012,367052.0,-9031.0,430934.0,18266.0,9235.0,0.011573,20121,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4900,VSC,4,2019,2070078.0,-552.0,1082398.0,110987.0,110435.0,0.035031,20194,0.030265
4901,VSC,1,2020,2141761.0,-1075.0,1029230.0,81263.0,80188.0,0.025288,20201,0.029151
4902,VSC,2,2020,2185842.0,-639.0,1016896.0,73815.0,73176.0,0.022848,20202,0.027461
4903,VSC,3,2020,2175866.0,-89.0,965396.0,86117.0,86028.0,0.027386,20203,0.025681


In [48]:
price = common.get_stock_price(stock_arr=hose_list, 
                               to_date='2021-04-30', 
                               from_date='2016-03-31', pivot_type=False, price=True)

In [49]:
oi = common.get_stock_oi(stock_arr=hose_list, to_date='2021-04-30', from_date='2015-01-01', pivot_type=False)

In [50]:
price_oi = pd.merge(price, oi, on=['date', 'code'], how='left')

In [51]:
price_oi['marcap'] = price_oi['close'] * 1000 * price_oi['oi']

In [52]:
price_oi['date'] = price_oi['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [53]:
price_oi['month'] = price_oi['date'].dt.month

In [54]:
price_oi.loc[(price_oi['month']!=price_oi['month'].shift(-1))|(
            price_oi['code']!=price_oi['code'].shift(-1)), 'choose'] = 1
price_oi_monthly = price_oi[price_oi['choose']==1].reset_index(drop=True).drop(columns=['choose', 'month'])

In [55]:
price_oi_monthly['date_lag_x'] = price_oi_monthly['date'].apply(lambda x: x-relativedelta(months=5))
price_oi_monthly['year'] = price_oi_monthly['date_lag_x'].dt.year
price_oi_monthly['quarter'] = price_oi_monthly['date_lag_x'].dt.month.apply(lambda x: ceil(x/3))


In [56]:
price_oi_fin_monthly = pd.merge(price_oi_monthly, data, on=['code', 'year', 'quarter'])

In [57]:
price_oi_fin_monthly['earnings yield'] = price_oi_fin_monthly['ebit'] / price_oi_fin_monthly['marcap']

In [58]:
price_oi_fin_monthly['pB'] = price_oi_fin_monthly['marcap'] / price_oi_fin_monthly['total equity']

In [59]:
price_oi_fin_monthly[price_oi_fin_monthly['code']=='AAA']

,date,code,close,oi,marcap,date_lag_x,year,quarter,total equity,interest expenses,fixed assets,ebt,ebit,roic,qrt_yr,roic_5yr_avg,earnings yield,pB
0,2016-03-31,AAA,9.64,49.499989,4.771799e+05,2015-10-31,2015,4,819485.0,-6474.0,636532.0,27233.0,20759.0,0.014257,20154,0.010377,0.043504,0.582292
1,2016-04-29,AAA,11.18,49.499989,5.534099e+05,2015-11-29,2015,4,819485.0,-6474.0,636532.0,27233.0,20759.0,0.014257,20154,0.010377,0.037511,0.675314
2,2016-05-31,AAA,14.47,51.899986,7.509928e+05,2015-12-31,2015,4,819485.0,-6474.0,636532.0,27233.0,20759.0,0.014257,20154,0.010377,0.027642,0.916420
3,2016-06-30,AAA,17.94,51.899986,9.310857e+05,2016-01-30,2016,1,845170.0,-6294.0,591620.0,32024.0,25730.0,0.017908,20161,0.010342,0.027634,1.101655
4,2016-07-29,AAA,21.28,51.899986,1.104432e+06,2016-02-29,2016,1,845170.0,-6294.0,591620.0,32024.0,25730.0,0.017908,20161,0.010342,0.023297,1.306757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,2020-12-31,AAA,14.40,221.759964,3.193343e+06,2020-07-31,2020,3,3966823.0,-49063.0,2383105.0,100022.0,50959.0,0.008025,20203,0.014617,0.015958,0.805013
58,2021-01-29,AAA,13.10,221.759964,2.905056e+06,2020-08-29,2020,3,3966823.0,-49063.0,2383105.0,100022.0,50959.0,0.008025,20203,0.014617,0.017541,0.732338
59,2021-02-26,AAA,14.30,221.759964,3.171167e+06,2020-09-26,2020,3,3966823.0,-49063.0,2383105.0,100022.0,50959.0,0.008025,20203,0.014617,0.016069,0.799422
60,2021-03-31,AAA,16.60,221.759964,3.681215e+06,2020-10-31,2020,4,4024162.0,-48081.0,2325027.0,69003.0,20922.0,0.003295,20204,0.014068,0.005683,0.914778


In [60]:
earnings = pd.pivot(price_oi_fin_monthly, 
                    index='date', columns='code', values='earnings yield').dropna(how='all', axis=1)

roic = pd.pivot(price_oi_fin_monthly, 
                index='date', columns='code', values='roic').dropna(how='all', axis=1)

pB = pd.pivot(price_oi_fin_monthly, 
                index='date', columns='code', values='pB').dropna(how='all', axis=1)

roic_5y = pd.pivot(price_oi_fin_monthly, 
                index='date', columns='code', values='roic_5yr_avg').dropna(how='all', axis=1)

In [61]:
roic_ranking = roic.copy(deep=True)
earnings_ranking = earnings.copy(deep=True)
pB_ranking = pB.copy(deep=True)
roic_5y_ranking = roic_5y.copy(deep=True)

for col in roic_ranking.columns:
    roic_ranking[col] = np.nan
    earnings_ranking[col] = np.nan
    pB_ranking[col] = np.nan
    roic_5y_ranking[col] = np.nan

In [62]:
roic_ranking = roic.rank(axis=1, ascending=False)
earnings_ranking = earnings.rank(axis=1, ascending=False)
pB_ranking = pB.rank(axis=1, ascending=True)
roic_5y_ranking = roic_5y.rank(axis=1, ascending=False)

In [63]:
combined_ranks = (roic_ranking + earnings_ranking + pB_ranking + roic_5y_ranking)/4

combined_ranks = combined_ranks.rank(axis=1, ascending=True)

In [64]:
price_monthly_pivot = price_oi_fin_monthly.pivot(index='date', columns='code', values='close')
monthly_return = price_monthly_pivot/price_monthly_pivot.shift(1)-1

In [65]:
combined_ranks = combined_ranks.dropna(how='all', axis=0)
combined_ranks = combined_ranks.dropna(how='all', axis=1)


In [66]:
# combined_ranks['count'] = combined_ranks.count(axis='columns')

# for col in combined_ranks.columns[:-1]:
#     combined_ranks[col] = combined_ranks[col]/combined_ranks['count']
    

In [67]:
combined_ranks

code,AAA,AMD,ANV,APG,ASM,BCG,BID,BMP,CCL,CII,...,VCB,VCI,VGC,VHC,VIC,VND,VNG,VNM,VPB,VSC
date,,,,,,,,,,,,,,,,,,,,,
2016-03-31,48.0,NaN,64.0,9.0,72.0,NaN,NaN,16.0,51.0,79.0,...,47.0,NaN,NaN,30.0,75.0,NaN,69.5,35.0,NaN,37.0
2016-04-29,49.0,NaN,62.0,7.0,73.5,NaN,NaN,14.5,48.0,79.0,...,46.0,NaN,NaN,31.0,75.0,NaN,68.5,36.0,NaN,34.0
2016-05-31,59.0,NaN,62.0,12.0,73.0,NaN,NaN,14.0,47.0,79.0,...,46.0,NaN,NaN,31.0,74.0,NaN,70.0,35.5,NaN,32.0
2016-06-30,53.0,NaN,67.5,74.0,69.0,NaN,NaN,5.0,46.0,75.0,...,38.0,NaN,NaN,17.5,52.0,NaN,81.0,25.5,NaN,41.0
2016-07-29,59.0,NaN,66.5,74.0,68.0,NaN,NaN,7.0,44.5,75.0,...,40.0,NaN,NaN,21.0,53.0,NaN,81.0,26.0,NaN,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31,60.5,88.5,70.5,58.5,78.0,74.0,65.0,13.0,58.5,101.0,...,57.0,56.0,68.0,23.5,111.0,10.0,112.0,39.0,9.0,32.0
2021-01-29,62.0,85.5,57.0,49.0,81.0,74.0,59.0,11.0,61.0,100.0,...,55.0,52.0,79.5,20.0,112.0,4.0,111.0,42.0,7.0,39.5
2021-02-26,65.0,88.0,58.5,43.0,75.5,79.0,58.5,9.0,68.0,100.0,...,54.5,52.0,82.5,18.0,112.0,4.0,110.0,38.0,10.5,32.0


In [68]:
### CHECK PERFORMANCE OF THE TOP OF SPECIFIC STOCKS (TOP 15 STOCKS, 10 STOCKS, ...)

positions = monthly_return.copy(deep=True)
for col in positions.columns:
    positions[col] = np.nan
    
for col in positions.columns:
    if col in combined_ranks.columns:
        positions[col] = combined_ranks[col].apply(lambda x: 1 if x<=5 else np.nan)
    else:
        positions = positions.drop(columns=col)
        
profit = monthly_return * positions.shift(1)

profit['avgProfit'] = profit.mean(axis=1)

profit['cumulative'] = (profit['avgProfit'] + 1).cumprod()

profit['numHolding'] = profit.count(axis=1)

import plotly.graph_objects as go
cumulative = profit['cumulative'].to_list()
benchmark = (monthly_return.mean(axis=1)+ 1).cumprod().tolist()
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=profit.index.tolist(), y=cumulative,
               mode='lines',
               name='cumulative'))
fig.add_trace(
    go.Scatter(x=profit.index.tolist(), y=benchmark,
               mode='lines',
               name='benchmark'))

fig.show()

In [70]:
profit

code,AAA,AMD,ANV,APG,ASM,AST,BCG,BID,BMP,BWE,...,VNG,VNM,VPB,VPG,VPI,VRE,VSC,avgProfit,cumulative,numHolding
date,,,,,,,,,,,,,,,,,,,,,
2016-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2016-04-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.040967,1.040967,7
2016-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.027329,1.069415,7
2016-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007548,1.077487,7
2016-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.116483,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.079124,1.162742,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.212687,NaN,NaN,NaN,NaN,0.175762,5.166513,7
2021-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.041963,5.383313,7
2021-02-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.173163,6.315506,7


In [73]:
a= combined_ranks.reset_index()
a = a.melt(id_vars=['date'], value_vars=a.columns[1:])
a[a['date']=='2016-03-31'][a['value']<=5]

C:\Users\ADMIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,date,code,value
992,2016-03-31,D2D,1.5
1860,2016-03-31,DXG,3.0
3968,2016-03-31,KSB,5.0
4712,2016-03-31,NTL,4.0
5022,2016-03-31,PHR,1.5


In [72]:
b = profit.reset_index()
b=b.melt(id_vars=['date'], value_vars=b.columns[1:-3])
b[b['date']=='2016-04-29'].dropna()

,date,code,value
1241,2016-04-29,D2D,0.040840
2171,2016-04-29,DXG,-0.067867
4713,2016-04-29,KSB,0.153417
5457,2016-04-29,NTL,-0.067416
5829,2016-04-29,PHR,0.145861
